In [21]:
# set up 
import pandas as pd

In [22]:
# load data files
school_resource_data = "Resources/schools_complete.csv"
student_resource_data = "Resources/students_complete.csv"

In [23]:
# Read data and store into pandas Data Frames
school_data = pd.read_csv(school_resource_data)
student_data = pd.read_csv(student_resource_data)

In [24]:
# Merge Data Frames
school_student_data = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_student_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [25]:
# District Summary
# --------------------------------------------
# Total Schools, Total Students, Total Budget
# Average Math Score, Average Reading Score
# Percentages Passing Math and Reading and the Overall Passing Rate
# Clean Column Headers
ss_data_cleanhead = school_student_data.rename(columns={
    "Student ID": "Student ID",
    "student_name": "Student Name",
    "gender":"Gender", 
    "grade": "Grade",
    "school_name": "School Name",
    "reading_score": "Reading Score",
    "math_score": "Math Score",
    "type": "School Type",
    "size": "Size",
    "budget": "Budget"
})

In [26]:
# Get values for summary data frame
total_schools = len(ss_data_cleanhead["School Name"].unique())
total_students = ss_data_cleanhead["Size"].sum()
total_budget = ss_data_cleanhead["Budget"].sum()

average_math = ss_data_cleanhead["Math Score"].mean()
average_reading = ss_data_cleanhead["Reading Score"].mean()

# Calculate passing rates

# Math
math_over_70 = ss_data_cleanhead.loc[ss_data_cleanhead["Math Score"] >69]
num_mathover_70 = math_over_70["Math Score"].count()
percent_pass_math = num_mathover_70/(len(ss_data_cleanhead["Math Score"]))

# Reading
read_over_70 = ss_data_cleanhead.loc[ss_data_cleanhead["Reading Score"] >69]
num_readover_70 = read_over_70["Reading Score"].count()
percent_pass_read = num_readover_70/(len(ss_data_cleanhead["Reading Score"]))

#Overall Passing
overall_passing = (percent_pass_math + percent_pass_read)/2


In [27]:
# District Summary DataFrame
# --------------------------------------------

district_summ_dict = {
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math],
    "Average Reading Score": [average_reading],
    "Percentage Passing Math": [percent_pass_math],
    "Percentage Passing Reading": [percent_pass_read],
    "Overall Passing Rate": [overall_passing]
}

district_summ_table = pd.DataFrame(district_summ_dict)
district_summ_table.style.format({
    'Total Students': '{:,.0f}'.format,
    'Total Budget': '${:,.0f}'.format,
    'Average Math Score': '{:,.2f}'.format,
    'Average Reading Score': '{:,.2f}'.format,
    'Percentage Passing Math': '{:,.2%}'.format,
    'Percentage Passing Reading': '{:,.2%}'.format,
    'Overall Passing Rate': '{:,.2%}'.format,

})

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading,Overall Passing Rate
0,15,"130,551,930","$82,932,329,558",78.99,81.88,74.98%,85.81%,80.39%


In [28]:
# School Summary
# ------------------------------
# Organize data by school name to get the following info:
#   School type, Total Students per School, Total School Budget
#   Total Budget per student at school, 
#   School's average math and reading scores
#   School's percent passing percentages for math and reading
#   Overall passing rates at the school


In [29]:
#set index and groupby school name, get/calculate summary table values
grouped_by_school = ss_data_cleanhead.set_index("School Name").groupby(["School Name"])
#
total_students_per_school = grouped_by_school.size()
school_type = grouped_by_school["School Type"].first()
#
schools_total_budget = grouped_by_school["Budget"].first()
budget_per_student = schools_total_budget/total_students_per_school
#
ave_math_per_school = grouped_by_school["Math Score"].mean()
ave_read_per_school = grouped_by_school["Reading Score"].mean()
#
groupby_passing_math = ss_data_cleanhead[ss_data_cleanhead["Math Score"]>=70].groupby(["School Name"]).size()
schools_percent_pass_math = groupby_passing_math/total_students_per_school
#
groupby_passing_read = ss_data_cleanhead[ss_data_cleanhead["Reading Score"]>=70].groupby(["School Name"]).size()
schools_percent_pass_read = groupby_passing_read/total_students_per_school
#
schools_overall_passing = (schools_percent_pass_math + schools_percent_pass_read)/2

In [31]:
# School Summary Data Frame
# ------------------------------

school_summ_dict = {
    "School Type": school_type,
    "Total Students": total_students_per_school,
    "Total Budget": schools_total_budget,
    "Total Budget Per Student": budget_per_student,
    "Average Math Score": ave_math_per_school,
    "Average Reading Score": ave_read_per_school,
    "Percentage Passing Math": schools_percent_pass_math,
    "Percentage Passing Reading": schools_percent_pass_read,
    "Overall Passing Rate": schools_overall_passing
}

school_summ_table = pd.DataFrame(school_summ_dict)

# Make a copy for display
school_summ_table_copy = school_summ_table.copy()
school_summ_table_copy
#school_summ_table_copy["Total Students"] = school_summ_table_copy["Total Students"].map("${:,.2f}".format)
#school_summ_table_copy["Total Budget"] = school_summ_table_copy["Total Budget"].map("${:,.2f}".format)
#school_summ_table_copy["Total Budget Per Student"] = school_summ_table_copy["Total Budget Per Student"].map("${:,.2f}".format)
#school_summ_table_copy["Percentage Passing Math"] = school_summ_table_copy["Total Budget Per Student"].map("{:,.2%}".format)
#school_summ_table_copy.index.name = None


,School Type,Total Students,Total Budget,Total Budget Per Student,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,0.666801,0.819333,0.743067
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.955867
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.733639
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,0.683096,0.792990,0.738043
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.952657
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.667530,0.808630,0.738080
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,0.925059,0.962529,0.943794
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.735002
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.660576,0.812224,0.736400


In [32]:
# Top Performing Schools
# ------------------------------

top_performing_schools = school_summ_table.sort_values(by="Overall Passing Rate", ascending=False)
top_performing_schools.head(5)

,School Type,Total Students,Total Budget,Total Budget Per Student,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.955867
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,0.932722,0.973089,0.952905
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,0.945946,0.959459,0.952703
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.952657
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,0.938677,0.965396,0.952037


In [33]:
# Bottom Performing Schools
# ------------------------------

bottom_performing_schools = school_summ_table.sort_values(by="Overall Passing Rate", ascending=True)
bottom_performing_schools.head(5)

,School Type,Total Students,Total Budget,Total Budget Per Student,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,0.663666,0.802201,0.732933
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.733639
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.735002
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.660576,0.812224,0.736400
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,0.683096,0.792990,0.738043


In [34]:
# Math Scores by Grade
# ------------------------------

math_nine = ss_data_cleanhead[ss_data_cleanhead["Grade"]=="9th"].groupby("School Name")["Math Score"].mean()
math_ten = ss_data_cleanhead[ss_data_cleanhead["Grade"]=="10th"].groupby("School Name")["Math Score"].mean()
math_eleven = ss_data_cleanhead[ss_data_cleanhead["Grade"]=="11th"].groupby("School Name")["Math Score"].mean()
math_twelve = ss_data_cleanhead[ss_data_cleanhead["Grade"]=="12th"].groupby("School Name")["Math Score"].mean()

by_grade_math_scores_dict ={
    "9th": math_nine,
    "10th": math_ten,
    "11th": math_eleven,
    "12th": math_twelve,
}

by_grade_math_df = pd.DataFrame(by_grade_math_scores_dict)
by_grade_math_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [35]:
# Reading Scores by Grade
# ------------------------------

read_nine = ss_data_cleanhead[ss_data_cleanhead["Grade"]=="9th"].groupby("School Name")["Reading Score"].mean()
read_ten = ss_data_cleanhead[ss_data_cleanhead["Grade"]=="10th"].groupby("School Name")["Reading Score"].mean()
read_eleven = ss_data_cleanhead[ss_data_cleanhead["Grade"]=="11th"].groupby("School Name")["Reading Score"].mean()
read_twelve = ss_data_cleanhead[ss_data_cleanhead["Grade"]=="12th"].groupby("School Name")["Reading Score"].mean()

by_grade_read_scores_dict ={
    "9th": read_nine,
    "10th": read_ten,
    "11th": read_eleven,
    "12th": read_twelve,
}

by_grade_read_df = pd.DataFrame(by_grade_read_scores_dict)
by_grade_read_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [60]:
# Scores by Budget
# ------------------------------
# Analyse data to get bins

student_budget_df = school_summ_table["Total Budget Per Student"]
#
print(student_budget_df.max())
print(student_budget_df.min())
print(student_budget_df.mean())
student_budget_df.sort_values(ascending= True)

655.0
578.0
620.0666666666667


School Name
Wilson High School       578.0
Holden High School       581.0
Cabrera High School      582.0
Wright High School       583.0
Shelton High School      600.0
Pena High School         609.0
Griffin High School      625.0
Bailey High School       628.0
Rodriguez High School    637.0
Thomas High School       638.0
Figueroa High School     639.0
Ford High School         644.0
Johnson High School      650.0
Hernandez High School    652.0
Huang High School        655.0
Name: Total Budget Per Student, dtype: float64

In [73]:
# Scores by Budget
# ------------------------------

budget_bins = [0, 590, 630, 645, 660]
b_bin_names = ["Spending Least: <590", "Spending Range Lower-Middle: 590-630", "Spending Range Middle-Upper: 630-645", "Spending Most: 645-660"]

scores_by_budget = school_summ_table.loc[:, ["Average Math Score",
                                            "Average Reading Score",
                                            "Percentage Passing Math",
                                            "Percentage Passing Reading",
                                            "Overall Passing Rate"]]
scores_by_budget["School's Spending Per Student"]= pd.cut(school_summ_table["Total Budget Per Student"],  budget_bins, labels=b_bin_names)

organized_scores_by_budget = scores_by_budget[["School's Spending Per Student",
                                               "Overall Passing Rate",
                                               "Average Math Score",
                                               "Average Reading Score",
                                               "Percentage Passing Math",
                                               "Percentage Passing Reading"]]

organized_scores_by_budget.sort_values(by="Overall Passing Rate", ascending = True)

,School's Spending Per Student,Overall Passing Rate,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading
School Name,,,,,,
Rodriguez High School,Spending Range Middle-Upper: 630-645,0.732933,76.842711,80.744686,0.663666,0.802201
Figueroa High School,Spending Range Middle-Upper: 630-645,0.733639,76.711767,81.158020,0.659885,0.807392
Huang High School,Spending Most: 645-660,0.735002,76.629414,81.182722,0.656839,0.813164
Johnson High School,Spending Most: 645-660,0.736400,77.072464,80.966394,0.660576,0.812224
Ford High School,Spending Range Middle-Upper: 630-645,0.738043,77.102592,80.746258,0.683096,0.792990
Hernandez High School,Spending Most: 645-660,0.738080,77.289752,80.934412,0.667530,0.808630
Bailey High School,Spending Range Lower-Middle: 590-630,0.743067,77.048432,81.033963,0.666801,0.819333
Holden High School,Spending Least: <590,0.943794,83.803279,83.814988,0.925059,0.962529
Shelton High School,Spending Range Lower-Middle: 590-630,0.948609,83.359455,83.725724,0.938671,0.958546


In [75]:
# Scores by School Size
# ------------------------------
# Analyse data to get bins
school_size_df = school_summ_table["Total Students"]
#
print(school_size_df.max())
print(school_size_df.min())
print(school_size_df.mean())
school_size_df.sort_values(ascending= True)

4976
427
2611.3333333333335


School Name
Holden High School        427
Pena High School          962
Griffin High School      1468
Thomas High School       1635
Shelton High School      1761
Wright High School       1800
Cabrera High School      1858
Wilson High School       2283
Ford High School         2739
Huang High School        2917
Figueroa High School     2949
Rodriguez High School    3999
Hernandez High School    4635
Johnson High School      4761
Bailey High School       4976
Name: Total Students, dtype: int64

In [76]:
# Scores by School Size
# ------------------------------

school_size_bins = [0, 1000, 2000, 3000, 5000]
ssize_bin_names = ["Small: <1000 Students", "Medium: 1000-2000", "Medium-Large: 2000-3000", "Largest: 3000-5000"]

scores_by_size = school_summ_table.loc[:, ["Average Math Score",
                                            "Average Reading Score",
                                            "Percentage Passing Math",
                                            "Percentage Passing Reading",
                                            "Overall Passing Rate"]]
scores_by_size["Total Students"]= pd.cut(school_summ_table["Total Students"],  school_size_bins, labels=ssize_bin_names)

organized_scores_by_size = scores_by_size[["Total Students",
                                               "Overall Passing Rate",
                                               "Average Math Score",
                                               "Average Reading Score",
                                               "Percentage Passing Math",
                                               "Percentage Passing Reading"]]

organized_scores_by_size.sort_values(by="Overall Passing Rate", ascending = True)


,Total Students,Overall Passing Rate,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading
School Name,,,,,,
Rodriguez High School,Largest: 3000-5000,0.732933,76.842711,80.744686,0.663666,0.802201
Figueroa High School,Medium-Large: 2000-3000,0.733639,76.711767,81.158020,0.659885,0.807392
Huang High School,Medium-Large: 2000-3000,0.735002,76.629414,81.182722,0.656839,0.813164
Johnson High School,Largest: 3000-5000,0.736400,77.072464,80.966394,0.660576,0.812224
Ford High School,Medium-Large: 2000-3000,0.738043,77.102592,80.746258,0.683096,0.792990
Hernandez High School,Largest: 3000-5000,0.738080,77.289752,80.934412,0.667530,0.808630
Bailey High School,Largest: 3000-5000,0.743067,77.048432,81.033963,0.666801,0.819333
Holden High School,Small: <1000 Students,0.943794,83.803279,83.814988,0.925059,0.962529
Shelton High School,Medium: 1000-2000,0.948609,83.359455,83.725724,0.938671,0.958546


In [88]:
# Scores by School Type
# ------------------------------

scores_by_type = school_summ_table[["School Type",
                                    "Overall Passing Rate",
                                    "Average Math Score",
                                    "Average Reading Score",
                                    "Percentage Passing Math",
                                    "Percentage Passing Reading"]]

scores_by_type = scores_by_type.groupby("School Type").mean()
scores_by_type

,Overall Passing Rate,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading
School Type,,,,,
Charter,0.951037,83.473852,83.896421,0.936208,0.965865
District,0.736738,76.956733,80.966636,0.665485,0.807991
